In [ ]:
# Imports
import os
import pickle
import numpy as np
rng_seed = 399
np.random.seed(rng_seed)
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from umap import UMAP
from configparser import ConfigParser

import plotly.offline as pyo
pyo.init_notebook_mode()

from importlib import reload
import eval_utils
import train_utils
import utils
reload(eval_utils)
reload(train_utils)
reload(utils)

from eval_utils import (
    get_word_vec,
    similar_words,
    create_embeddings_of_train_weight_checkpoints,
    visualize_embeddings_over_time,
    plot_word_relationships_2d,
    plot_word_vectors,
    evaluate_model_questions_words
)
from train_utils import get_model_checkpoint_filepaths
from utils import load_model
from word2vec import Word2vec

In [ ]:
# Constants
checkpoints_dir = "checkpoints"
checkpoint_filepaths_dict = get_model_checkpoint_filepaths(
    checkpoints_dir=checkpoints_dir,
    model_name="word2vec_sgns",
    dataset_name="enwiki",
)

In [ ]:
# Load model training configuration
model_training_conf = ConfigParser()
model_training_conf.read(checkpoint_filepaths_dict["model_training_conf_filepath"])

In [ ]:
# Load words and create word to int lookup dict
with open(checkpoint_filepaths_dict["train_words_filepath"], "r") as file:
    words = np.array(file.read().split("\n"))
word_to_int = {word: i for i, word in enumerate(words)}

In [ ]:
# Load last model
# word2vec = load_model(checkpoint_filepaths_dict['model_filepaths'][-1])
# word2vec._max_window_size = 5
# word2vec.save_model_training_conf("checkpoints/word2vec_sgns_enwiki.conf", 5)

In [ ]:
# Get vocabulary size, embedding dimension, word to int dictionary and words used in the models
vocab_size = model_training_conf["MODELCONFIG"].getint("vocab_size")
embedding_dim = model_training_conf["MODELCONFIG"].getint("embedding_dim")

In [ ]:
# Get target embedding weights of last model
last_embedding_weights = np.load(
    checkpoint_filepaths_dict["intermediate_embedding_weight_filepaths"][-1],
    mmap_mode="r",
)

## Visualize training over the course of epochs

In [ ]:
# Due to computational limitations, we only visualize the top 1000 most common words.
vis_embeddings_vocab_size = 10000

In [ ]:
# Create embeddings of word embeddings from all train checkpoints
umap_embeddings_over_time, cluster_labels_over_time = create_embeddings_of_train_weight_checkpoints(
    model_weights_filepaths=checkpoint_filepaths_dict["intermediate_embedding_weight_filepaths"],
    vocab_size=vis_embeddings_vocab_size,
    embedding_dim=embedding_dim,
    clusterer=KMeans(n_clusters=10, random_state=rng_seed),
    transformer=UMAP(n_components=3, random_state=rng_seed)
)

In [ ]:
# Visualize training
visualize_embeddings_over_time(
    transformed_word_embeddings=umap_embeddings_over_time,
    cluster_labels=cluster_labels_over_time,
    vocab_size=vis_embeddings_vocab_size,
    words=words
)

## Find similar words

In [ ]:
# Find closest word to
similar_words(
    positive_words=["man"],
    weights=last_embedding_weights,
    word_to_int=word_to_int,
    words=words,
    top_n=10,
    vocab_size=10000
)

In [ ]:
# Test similarities
similar_words(
    positive_words=["france", "mogadishu"],
    negative_words=["paris"],
    weights=last_embedding_weights,
    word_to_int=word_to_int,
    words=words,
    top_n=10,
    vocab_size=100000
)

## Plot word relationships

In [ ]:
# Create 2D PCA embeddings of last model
embedding_weights_2d_pca = PCA(n_components=2, random_state=rng_seed).fit_transform(last_embedding_weights)

In [ ]:
pairs = [
    ('man', 'woman'),
    ('king', 'queen')
]
plot_word_relationships_2d(
    pairs,
    embedding_weights_2d_pca,
    word_to_int,
    x_label="PC 1",
    y_label="PC 2"
)

In [ ]:
# Create 2D UMAP embeddings of last model
embedding_weights_2d_umap = UMAP(n_components=2, random_state=rng_seed).fit_transform(last_embedding_weights)

In [ ]:
# Plot words one through nine to check for cirular shape
zero_to_nine = [
    'zero',
    'one',
    'two',
    'three',
    'four',
    'five',
    'six',
    'seven',
    'eight',
    'nine'
]
plot_word_vectors(
    zero_to_nine,
    embedding_weights_2d_umap,
    word_to_int,
    x_label="UMAP 1",
    y_label="UMAP 2"
)

## Validate model on "questions-words" pairs

In [ ]:
evaluate_model_questions_words(
    "data/questions-words.pickle",
    last_embedding_weights,
    word_to_int,
    words,
    top_n=5
)